In [1]:
# Import required modules
import numpy as np
import pandas as pd
import seaborn as sns
import shapefile as shp
import matplotlib.pyplot as plt

In [2]:
# Initialise visualisation set
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(14, 10))

In [3]:
# Open the vector map
shp_path = "./data/County.shp"
sf = shp.Reader(shp_path)
len(sf.shapes())

47

In [4]:
# Explore the shapefile
sf.records()

[Record #0: [1, 5.677, 15.047, 2.0, 1.0, 'Turkana', 15.0468376942, 5.67698507035],
 Record #1: [2, 6.177, 11.974, 3.0, 2.0, 'Marsabit', 11.9741649875, 6.1768307404],
 Record #2: [3, 2.117, 7.355, 4.0, 3.0, 'Mandera', 7.35515432979, 2.11719606749],
 Record #3: [4, 4.61, 9.838, 5.0, 4.0, 'Wajir', 9.83840789987, 4.60958923426],
 Record #4: [5, 0.74, 5.03, 6.0, 5.0, 'West Pokot', 5.03027088343, 0.740480579545],
 Record #5: [6, 1.713, 8.311, 7.0, 6.0, 'Samburu', 8.31101343379, 1.71301446473],
 Record #6: [7, 2.06, 10.181, 8.0, 7.0, 'Isiolo', 10.181410715, 2.05982040515],
 Record #7: [8, 0.877, 5.964, 9.0, 8.0, 'Baringo', 5.96426557677, 0.877177301723],
 Record #8: [9, 0.245, 4.064, 10.0, 9.0, 'Keiyo-Marakwet', 4.06352699188, 0.245207562504],
 Record #9: [10, 0.202, 2.387, 11.0, 10.0, 'Trans Nzoia', 2.38726517511, 0.201982889373],
 Record #10: [11, 0.246, 3.076, 12.0, 11.0, 'Bungoma', 3.0759439997, 0.245741265658],
 Record #11: [12, 3.642, 11.281, 13.0, 12.0, 'Garissa', 11.2811263395, 3.6418

In [5]:
# Check one County
sf.record(0)[5]

'Turkana'

In [6]:
# Function to Convert the shapefile to a Pandas DataFrame
def read_shapefile(sf):
    # Fetch the headers from the shapefile
    fields = [x[0] for x in sf.fields][1:]
    # Fetch the coords from the shapefile
    records = [list(i) for i in sf.records()]
    shps = [s.points for s in sf.shapes()]
    # Create a DataFrame
    df = pd.DataFrame(columns=fields, data=records)
    # Add the coords to the DataFrame
    df = df.assign(coords=shps)
    return df

In [7]:
# Read the shapefile
df = read_shapefile(sf)
df.shape

(47, 9)

In [ ]:
# Sample of the DataFrame
df.head(46)

In [ ]:
# Plot the shapefile with matplotlib
def plot_shape(id, s=None):
    plt.figure()
    # Plot axes
    ax = plt.axes()
    ax.set_aspect('equal')
    # Store id number to be used
    shape_ex = sf.shape(id)
    # Create arrays with 0 in place of each element - len(shape_ex.points) Rows and 1 Column
    x_lon = np.zeros((len(shape_ex.points), 1)) # x-coordinates, longitude
    # Create arrays with 0 in place of each element - len(shape_ex.points) Rows and 1 Column
    y_lat = np.zeros((len(shape_ex.points), 1)) # y-coordinates, latitude
    for  ip in range(len(shape_ex.points)):
        x_lon[ip] = shape_ex.points[ip][0]
        y_lat[ip] = shape_ex.points[ip][1]
    # Plot the derived cordinates on the map
    plt.plot(x_lon, y_lat)
    x0 = np.mean(x_lon)
    y0 = np.mean(y_lat)
    plt.text(x0, y0, s, fontsize=10)
    # Use bounding box to set plot limits
    plt.xlim(shape_ex.bbox[0], shape_ex.bbox[2])
    return x0, y0

In [ ]:
# Enter the name of a county to plot
county_name = df['COUNTY'][2]
county_id = 2
plot_shape(county_id, county_name)
sf.shape(county_id)


In [ ]:
# Plot the entire map using matplotlib
def plot_map(sf, x_lim = None, y_lim = None, figsize = (20,15)):
    plt.figure(figsize = figsize)
    plt.title('Kenya - All Counties')
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    id=0
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.plot(x, y, 'k')

        if (x_lim == None) & (y_lim == None):
            x0 = np.mean(x)
            y0 = np.mean(y)
            plt.text(x0, y0, shape.record[5], fontsize=10)
        id = id+1

    if (x_lim != None) & (y_lim != None):
        plt.xlim(x_lim)
        plt.ylim(y_lim)

In [ ]:
plot_map(sf)

In [ ]:
# Plot A zoomed in map
#x_lim = [35, 36]
#y_lim = [1, 2]
#plot_map(sf, x_lim, y_lim)

In [ ]:
# Highlight a county in the map
def plot_map_fill(id, sf, x_lim = None,
                    y_lim = None,
                    figsize = (11,9),
                    color = 'r'):

    plt.figure(figsize = figsize)
    fig, ax = plt.subplots(figsize = figsize)
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        ax.plot(x, y, 'k')

    shape_ex = sf.shape(id)
    x_lon = np.zeros((len(shape_ex.points),1))
    y_lat = np.zeros((len(shape_ex.points),1))
    for ip in range(len(shape_ex.points)):
        x_lon[ip] = shape_ex.points[ip][0]
        y_lat[ip] = shape_ex.points[ip][1]
    ax.fill(x_lon,y_lat, color)

    if (x_lim != None) & (y_lim != None):
        plt.xlim(x_lim)
        plt.ylim(y_lim)


In [ ]:
#plot_map_fill(0, sf, x_lim, y_lim, color=’y’)
plot_map_fill(12, sf, x_lim=None, y_lim=None, color='g')

In [ ]:
def plot_map_fill_multiples_ids(title, county, sf,
                                    x_lim = None,
                                    y_lim = None,
                                    figsize = (11,9),
                                    color = 'r'):

    plt.figure(figsize = figsize)
    fig, ax = plt.subplots(figsize = figsize)
    fig.suptitle(title, fontsize=16)
    for shape in sf.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        ax.plot(x, y, 'k')
    for id in county:
        shape_ex = sf.shape(id)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        ax.fill(x_lon,y_lat, color)
        x0 = np.mean(x_lon)
        y0 = np.mean(y_lat)
        plt.text(x0, y0, id, fontsize=10)
    if (x_lim != None) & (y_lim != None):
        plt.xlim(x_lim)
        plt.ylim(y_lim)

In [ ]:
# Plot the map with multiple ids
county_ids = [4,7,8,12,17,23,31]
plot_map_fill_multiples_ids('Bedroom Ya Ankoo', county_ids, sf, x_lim=None, y_lim=None, color='y')